#Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Concatenate, Dense, Flatten
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import requests
from io import BytesIO
import cv2

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential

#Importing the train and test dataset

In [ ]:
!gdown 174YvQO7mdg0tXF_e-gLkd0zpcKdwNbSz

Downloading...
From: https://drive.google.com/uc?id=174YvQO7mdg0tXF_e-gLkd0zpcKdwNbSz
To: /content/content_simulation_train.xlsx
100% 56.2M/56.2M [00:00<00:00, 103MB/s]


In [ ]:
df = pd.read_excel("/content/content_simulation_train.xlsx")

In [ ]:
df.columns

Index(['id', 'date', 'likes', 'content', 'username', 'media',
       'inferred company'],
      dtype='object')

In [ ]:
import re
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

def get_counts(df):
  mention = "<mention>"
  df['mention_count'] = df['content'].apply(lambda x: x.split().count(mention))
  #
  hyperlink = "<hyperlink>"
  df["hyperlink_count"] = df['content'].apply(lambda x: x.split().count(hyperlink))
  #
  df['hashtag_count'] = df['content'].apply(lambda x: sum([1 for word in x.split() if word.startswith('#')]))
  df["word_count"] = df['content'].apply(lambda x: sum([1 for word in x.split()]))
  df['content'] = df['content'].str.replace('<mention>', '')
  df['content'] = df['content'].str.replace('<hyperlink>', '')
  df['content'] = df['content'].str.replace('<mention>', '')
  df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('#')]))
  return df

def get_media_urls(df):
  df['image_url'] = df['media'].str.extract(r"fullUrl='(.*?)'")
  import re
  df['video_url'] = df['media'].apply(lambda x: re.findall(r"url='(.*?)'", x)[1] if 'variants' in x and len(re.findall(r"url='(.*?)'", x)) > 1 else None)
  import ast
  df['thumbnail_url'] = df['media'].str.extract(r"thumbnailUrl='(.*?)'")
  df['duration'] = df['media'].apply(lambda x: float(re.search(r"duration=([\d.]+)", x).group(1)) if 'duration' in x else None)
  df['views'] = df['media'].apply(lambda x: int(re.search(r"views=(\d+)", x).group(1)) if re.search(r"views=(\d+)", x) else None)

  return df
def get_url(row):
  if isinstance(row['image_url'],str):
    return row['image_url']
  else:
    return row['thumbnail_url']

def infcorrection(df):
  df['inferred_company'] = df['inferred company']
  df.drop('inferred company',axis = 1)
  return df

import math
def get_views_duration(df):

  views = list(df['views'])
  result_list = [int(value) if not math.isnan(value) else 0 for value in views]
  df['views'] = result_list/df['views'].max()
  duration = list(df['duration'])
  result_list = [int(value) if not math.isnan(value) else 0 for value in duration]
  df['duration'] = result_list/df['duration'].max()

  return df

from datetime import datetime

# Assuming you have a DataFrame named df with a 'datetime' column
# Replace 'your_data.csv' with the actual file path or URL of your data
# df = pd.read_csv('your_data.csv')

# Convert 'datetime' column to datetime format if not already


# Function to create sine and cosine embeddings for time and date
def create_time_embeddings(df, column_name):

    seconds_in_day = 24 * 60 * 60
    seconds_in_year = 365.25 * 24 * 60 * 60

    # Calculate normalized values
    seconds_of_day = (df[column_name].dt.hour * 3600 + df[column_name].dt.minute * 60 + df[column_name].dt.second) / seconds_in_day
    seconds_of_year = (df[column_name] - df[column_name].min()).dt.total_seconds() / seconds_in_year

    # Create embeddings
    df['day_sin'] = np.sin(2 * np.pi * seconds_of_day)
    df['day_cos'] = np.cos(2 * np.pi * seconds_of_day)
    df['year_sin'] = np.sin(2 * np.pi * seconds_of_year)
    df['year_cos'] = np.cos(2 * np.pi * seconds_of_year)

    df['week_sin'] = np.sin((df['day']*2*np.pi)/7)
    df['week_cos'] = np.cos((df['day']*2*np.pi)/7)
    # df['week_sin'] =
    # df['week_cos'] =
    return df

# Apply the function to create embeddings for 'datetime' column
# df = create_time_embeddings(df, 'datetime')

# from datetime import datetime
def uptime(date):
  ref = '2023-11-21 00:00:00'
  refdate = pd.to_datetime(ref)
  reftime = refdate.timestamp()
  return reftime - date.timestamp()
def get_uptime(df):
  df['uptime'] = df['datetime'].apply(uptime)
  return df

def normalize_uptime(df):
  uptime = list(df['uptime'])
  upt = [int(value) if not math.isnan(value) else 0 for value in uptime]
  df['uptime'] = upt/df['uptime'].max()
  return df

def getweekday(date):
  return date.weekday()

def get_weekday_df(df):
  df['datetime'] = pd.to_datetime(df['date'])
  df['day'] = df['datetime'].apply(getweekday)
  return df

In [ ]:
def master_eda(df):
  df = df.dropna(subset=['content'])
  df = get_counts(df)
  df['content'] = df['content'].apply(remove_emojis)
  df = get_media_urls(df)
  df['url'] = df.apply(get_url,axis = 1)
  df = infcorrection(df)
  df = get_views_duration(df)
  df = get_weekday_df(df)
  df = create_time_embeddings(df,'datetime')
  df = get_uptime(df)
  df = normalize_uptime(df)
  return df[['id','likes', 'content', 'username', 'mention_count', 'hyperlink_count', 'hashtag_count',
       'word_count', 'duration',
       'views', 'url', 'inferred_company', 'day_sin', 'day_cos',
       'year_sin', 'year_cos', 'uptime', 'week_sin', 'week_cos']]







In [ ]:
df = master_eda(df)

In [ ]:
df = df.sample(n=50000, random_state=42)

In [ ]:
df.head()

,id,likes,content,username,mention_count,hyperlink_count,hashtag_count,word_count,duration,views,url,inferred_company,day_sin,day_cos,year_sin,year_cos,uptime,week_sin,week_cos
4941,4942,94,'The preacher loved his big moment – and the c...,Independent,0,2,0,16,0.000000,0.000000,https://pbs.twimg.com/media/DdqTf61V4AAmeXl?fo...,independent,-0.983872,0.178874,0.672191,-0.740378,0.934989,-0.781831,0.623490
51775,51776,23,Walk this way to see the musicians from the 19...,AMCTheatres,0,2,0,20,0.000000,0.000000,https://pbs.twimg.com/media/D2nsg4xVAAA3UQi?fo...,amc,-0.241640,0.970366,0.993547,0.113417,0.790718,0.781831,0.623490
115253,115254,187,Now that looks quite a-peeling! with the all n...,NokiamobileIN,0,1,1,14,0.033337,0.000007,https://pbs.twimg.com/ext_tw_video_thumb/10542...,nokia,0.772872,-0.634562,-0.938959,0.344028,0.863091,0.000000,1.000000
299321,299322,0,Happy birthday to the King—Elvis Presley—born ...,AAASouthJersey,0,1,0,29,0.000000,0.000000,https://pbs.twimg.com/media/DwZ3OjuXQAIUJr4?fo...,aaa,-0.968148,-0.250380,0.127872,0.991791,0.826649,0.781831,0.623490
173570,173571,1,"4 Persons Die, 6 Injured In Niger Auto Crash",IndependentNGR,0,2,0,11,0.000000,0.000000,https://pbs.twimg.com/media/EeRO7pqVAAASHMf?fo...,independent,-0.955085,-0.296333,-0.487038,-0.873381,0.561535,-0.433884,-0.900969


Normalize ze data

#Importing the encoder for text embeddings

##Text encoder

In [ ]:
# Load the Universal Sentence Encoder from TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

##Importing the RESNET model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions

# Load the ResNet-101 model pretrained on ImageNet
res_model = ResNet50(weights='imagenet', include_top=True)

for layer in res_model.layers:
  layer.trainable = False

img_model = Model(inputs = res_model.input,outputs = res_model.layers[-2].output)
img_model.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                       

##NIMA

In [ ]:
!git clone https://github.com/titu1994/neural-image-assessment

Cloning into 'neural-image-assessment'...
remote: Enumerating objects: 178, done.
remote: Total 178 (delta 0), reused 0 (delta 0), pack-reused 178
Receiving objects: 100% (178/178), 21.51 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [ ]:
%cd neural-image-assessment/

/content/neural-image-assessment


In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf



base_nima_model = MobileNet((None, None, 3), alpha=1, include_top=False, pooling='avg', weights=None)
x = Dropout(0.75)(base_nima_model.output)
x = Dense(10, activation='softmax')(x)

nima_model = Model(base_nima_model.input, x)
nima_model.load_weights('weights/mobilenet_weights.h5')


def get_score(img):
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)

    scores = nima_model.predict(x, batch_size=1, verbose=0)

    return np.array(scores[0])

In [ ]:
nima_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizati  (None, None, None, 32)    128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, None, None, 32)    288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, None, None, 32)    128 

#Creating a Regression Generator

In [ ]:
import random, glob, os
import numpy as np
from tqdm.notebook import tqdm
import typing
try :
    from keras.utils import Sequence #   sequence =  keras.utils.Sequence
except:
    from keras.utils.all_utils import Sequence

# ==================================================
class RegressionGenerator(Sequence):
    '''
    Dataset generator for Regression.
    '''
    def __init__(self, df_X : pd.DataFrame, df_Y : pd.DataFrame, width : int = 224, height : int = 224,
                 channels : int = 3, embedding_dim : int = 512, batch_size : int = 32,
                 encoder : Model = embed,
                 features : typing.List = ['content','url','mention_count','hashtag_count', 'day_sin', 'day_cos', 'week_sin', 'week_cos', 'year_sin', 'year_cos','uptime']) :
        """
        It is expected that the dataframe has already been split. As we are not generating any derivative data,
        there should be not need for different 'types' of generator. And I am lazy.
        """
        self.width, self.height, self.channels = width, height, channels  # Adjust as needed
        self.embedding_dim = embedding_dim  # Adjust as needed
        self.x_df = df_X[features]
        self.y_df = df_Y
        self.features = features
        self.batch_size = batch_size
        self.encoder = encoder
    # ==================================
    def __getitem__(self, index):
        # Disabling 4 threads for now
        batch_x = self.x_df[index*self.batch_size:(index+1)*self.batch_size]
        batch_y = self.y_df[index*self.batch_size:(index+1)*self.batch_size].to_numpy(dtype=float)
        contents = [] #!
        images = []
        others = []

        for index, row in batch_x.iterrows():
          contents.append(row['content'])
          img = self.read_data(row['url'])
          images.append(img)
          nima_score = get_score(img)
          # iipa_score = np.array(getiipa(img,iipa_model))
          # iipa_score = iipa_score.reshape(1)
          others.append(np.concatenate((nima_score,np.array(row[['mention_count','hashtag_count','day_sin','day_cos', 'week_sin', 'week_cos', 'year_sin', 'year_cos', 'uptime']]))))
        images = tf.convert_to_tensor(images)
        others = tf.convert_to_tensor(others)
        contents = tf.convert_to_tensor(self.encoder(contents))

        return [contents, images, others ], batch_y
#

    def __len__(self):
      return len(self.x_df)//self.batch_size

    def read_data (self, url) :
        '''
        Return the np.array of image @url
        '''
        response = requests.get(url, stream=True)
    # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Convert the image content to a NumPy array
            image_array = np.asarray(bytearray(response.content), dtype=np.uint8)

            # Decode the NumPy array to an OpenCV image
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

            # Resize the image to the desired dimensions
            image = cv2.resize(image, (self.width, self.height))

            # Normalize pixel values to the range [0, 1]
            image = image / 255.0

            # Return the preprocessed image
            return image


        return np.zeros((self.width, self.height, self.channels))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, df['likes'], test_size=0.2, random_state=42)

In [ ]:
train_features = ['content', 'mention_count', 'hashtag_count', 'url', 'day_sin','day_cos', 'week_sin', 'week_cos', 'year_sin', 'year_cos','uptime']

In [ ]:
training_gen = RegressionGenerator(X_train[train_features], y_train)
val_gen = RegressionGenerator(X_test[train_features], y_test)

#Model Regressors

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Reshape, Flatten, MaxPooling1D

In [ ]:
def text_processor(embed_dim: int = 512, params=[(4, 5), (2, 3)]):
    input_tensor = Input(shape=(embed_dim, 1))

    x = input_tensor

    # Replace Dense layers with 1D convolutional layers
    for filters, kernel_size in params:
        x = Conv1D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    output_tensor = x

    model = tf.keras.Model(input_tensor, output_tensor)
    return model

tp = text_processor()
tp.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 512, 4)            24        
                                                                 
 batch_normalization (Batch  (None, 512, 4)            16        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512, 4)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 512, 2)            26        
                                                                 
 batch_normalization_1 (Bat  (None, 512, 2)            8         
 chNormalization)                                          

In [ ]:
def other_processor(feature_dim : int = 19, params = [(8, 3), (2, 5)]):
  input_tensor = Input(shape = (feature_dim, 1))
  x = input_tensor
  for filters, kernel_size in params:
      x = Conv1D(filters, kernel_size, padding='same')(x)
      x = BatchNormalization()(x)

  x = MaxPooling1D(pool_size=2)(x)
  x = Flatten()(x)
  output_tensor = x
  model = tf.keras.Model(input_tensor, output_tensor)
  return model

op = other_processor()
op.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 19, 1)]           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 19, 8)             32        
                                                                 
 batch_normalization_2 (Bat  (None, 19, 8)             32        
 chNormalization)                                                
                                                                 
 conv1d_3 (Conv1D)           (None, 19, 2)             82        
                                                                 
 batch_normalization_3 (Bat  (None, 19, 2)             8         
 chNormalization)                                                
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 9, 2)              0   

In [ ]:
def image_processor(img_dim : np.array = (224,224,3), dense_dim : int = 128):
  input_tensor = img_model.input
  x = input_tensor
  x = BatchNormalization()(img_model.output)
  output_tensor = Dense(dense_dim, activation = 'relu')(x)
  model = tf.keras.Model(input_tensor, output_tensor)
  return model


In [ ]:
# def image_processor(img_dim=np.array((224, 224, 3)), params=[(8, 5),(2,3)]):
#     input_tensor = img_model.input

#     x = input_tensor

#     # Replace Dense layers with 1D convolutional layers
#     for filters, kernel_size in params:
#         x = Conv1D(filters, kernel_size, padding='same')(x)
#         x = BatchNormalization()(x)
#         x = Activation('relu')(x)

#     x = MaxPooling1D(pool_size=2)(x)
#     x = Flatten()(x)
#     output_tensor = x
#     model = tf.keras.Model(input_tensor, output_tensor)
#     return model

# im = image_processor()
# im.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 2048, 1)]         0         
                                                                 
 conv1d_5 (Conv1D)           (None, 2048, 8)           48        
                                                                 
 batch_normalization_5 (Bat  (None, 2048, 8)           32        
 chNormalization)                                                
                                                                 
 activation_3 (Activation)   (None, 2048, 8)           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 2048, 2)           50        
                                                                 
 batch_normalization_6 (Bat  (None, 2048, 2)           8         
 chNormalization)                                          

In [ ]:
def regression_model(embed_dim : int = 512, img_dim : typing.Tuple = (224,224,3), params = [(256, 'relu'),(128,'relu'), (1, 'relu')]):
  img_proc = image_processor(img_dim)
  text_proc = text_processor(embed_dim)
  other_proc = other_processor()

  input_tensor = tf.keras.layers.concatenate([text_proc.output, img_proc.output, other_proc.output])

  x = input_tensor
  for dim, activ in params:
    x = Dense(dim, activation = activ)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

  output_tensor = x
  model = tf.keras.Model([text_proc.input, img_proc.input, other_proc.input], output_tensor)

  return model

In [ ]:
reg  = regression_model()

In [ ]:
reg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3), loss='mean_squared_error', metrics=['mae'])

In [ ]:
reg.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

#Run Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = 'model_checkpoint.h5'  # Path to save the checkpoint
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',  # Monitor validation loss
                             save_best_only=False,  # Save the best model
                             mode='min',  # Mode to determine best model (minimize loss)
                             verbose=1)  # Verbosity level

hist = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator,
                    callbacks=[checkpoint])

In [ ]:
history = reg.fit(training_gen, validation_data = val_gen, epochs = 10,callbacks = [checkpoint])

Epoch 1/10
   9/1250 [..............................] - ETA: 7:04:36 - loss: 31402904.0000 - mae: 966.5695

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Specify the filepath for saving the training history
history_filepath = 'training_history.txt'

# Create a ModelCheckpoint callback to save the model and training history
checkpoint = ModelCheckpoint(filepath=history_filepath,
                             monitor='val_loss',  # Monitor a relevant metric
                             save_best_only=False,  # Save only the best model
                             save_weights_only=False,  # Save the entire model
                             mode='min',  # Save when the monitored metric decreases
                             verbose=1)  # Show a message when saving

# Save the final model weights separately (optional)
reg.save_weights('final_model_weights.h5')

# Save the final training history separately (optional)
with open('final_training_history.txt', 'w') as file:
    file.write(str(history.history))


In [ ]:
class SaveHistoryAndModel(Callback):
    def __init__(self, filename='training_history.txt'):
        super(SaveHistoryAndModel, self).__init__()
        self.filename = filename

    def on_epoch_end(self, epoch, logs=None):
        with open(self.filename, 'a') as file:
            file.write(f"Epoch {epoch + 1} - ")
            for key, value in logs.items():
                file.write(f"{key}: {value:.4f} ")
            file.write("\n")

model_filepath = 'model_checkpoint.h5'
history_filepath = 'training_history.txt'


In [ ]:
checkpoint = ModelCheckpoint(filepath=model_filepath,
                             monitor='val_loss',  # Monitor a relevant metric
                             save_best_only=True,  # Save only the best model
                             save_weights_only=False,  # Save the entire model
                             mode='min',  # Save when the monitored metric decreases
                             verbose=1)  # Show a message when saving


In [ ]:
save_history_and_model = SaveHistoryAndModel(filename=history_filepath)

history = reg.fit(training_gen,
                    epochs=10,
                    validation_data=val_gen,
                    callbacks=[checkpoint, save_history_and_model])
